# OneOffice - Local Testing

Notebook này cho phép chạy OneOffice pipeline từ local, không cần gọi Cloud Run.

**⚠️ QUAN TRỌNG - Đồng bộ với CloudRun:**
- Notebook này import trực tiếp từ code chính (`src/features/one_office/`)
- Khi code trên CloudRun thay đổi, notebook tự động sử dụng code mới
- Không cần copy/paste code - chỉ cần chạy lại notebook sau khi code thay đổi

**Flow:**
1. Extract: Lấy dữ liệu personnel từ 1Office API
2. Load: Upload raw JSON lên GCS (backup) và load vào BigQuery (snapshot)

**Sử dụng khi:**
- Test pipeline trước khi deploy
- Debug issues
- Chạy snapshot thủ công cho ngày cụ thể
- Test với credentials local


In [1]:
import os
import sys
from datetime import datetime, date
from dotenv import load_dotenv

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..'))
sys.path.insert(0, project_root)

# Load environment variables từ project root
env_path = os.path.join(project_root, '.env')
load_dotenv(dotenv_path=env_path)

print(f"Project root: {project_root}")
print(f"Python path: {sys.path[0]}")
print(f"Loading .env from: {env_path}")
print(f".env file exists: {os.path.exists(env_path)}")


Project root: d:\Atino\extension\etl-api-bigquery
Python path: d:\Atino\extension\etl-api-bigquery
Loading .env from: d:\Atino\extension\etl-api-bigquery\.env
.env file exists: True


In [2]:
# Import các module từ code chính
# Lưu ý: Notebook import trực tiếp từ src/features/one_office/ nên luôn đồng bộ với code CloudRun
from src.features.one_office.components.extractor import OneOfficeExtractor
from src.features.one_office.components.loader import OneOfficeLoader
from src.features.one_office.pipeline import OneOfficePipeline
from src.config import Settings
import logging

# ⚠️ QUAN TRỌNG: Reload Settings sau khi load_dotenv() để đọc env vars mới
# Settings được khởi tạo ở module level nên cần reload sau khi load .env
settings = Settings()

# Cấu hình logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ Đã import tất cả modules thành công")
print(f"✅ Settings reloaded - ONEOFFICE_ACCESS_TOKEN: {'✅ Có' if settings.oneoffice_access_token else '⚠️ Không có'}")


✅ Đã import tất cả modules thành công
✅ Settings reloaded - ONEOFFICE_ACCESS_TOKEN: ✅ Có


## Cấu hình

Load cấu hình từ environment variables và credential files.

**Lưu ý về Credentials:**
- 1Office API credentials: Cần set `ONEOFFICE_ACCESS_TOKEN` trong environment variable hoặc .env file
- BigQuery credentials: Sử dụng Application Default Credentials (ADC) hoặc service account key
- GCS bucket: Cấu hình trong `BRONZE_BUCKET` environment variable (dùng cho backup raw JSON)


In [3]:
# ⚠️ QUAN TRỌNG: Reload modules sau khi sửa code
# Chạy cell này trước khi chạy pipeline để đảm bảo code mới nhất được sử dụng
import importlib
import src.features.one_office.components.extractor
import src.features.one_office.components.loader
import src.features.one_office.pipeline
import src.config

# Reload modules
importlib.reload(src.config)
importlib.reload(src.features.one_office.components.extractor)
importlib.reload(src.features.one_office.components.loader)
importlib.reload(src.features.one_office.pipeline)

# Re-import sau khi reload
from src.config import Settings
from src.features.one_office.components.extractor import OneOfficeExtractor
from src.features.one_office.components.loader import OneOfficeLoader
from src.features.one_office.pipeline import OneOfficePipeline

# Reload Settings để đảm bảo đọc env vars mới nhất
settings = Settings()

# Re-initialize components với code mới
extractor = OneOfficeExtractor()
loader = OneOfficeLoader()
pipeline = OneOfficePipeline()

print("✅ Đã reload modules và khởi tạo lại components thành công")
print(f"✅ Settings reloaded - ONEOFFICE_ACCESS_TOKEN: {'✅ Có' if settings.oneoffice_access_token else '⚠️ Không có'}")


✅ Đã reload modules và khởi tạo lại components thành công
✅ Settings reloaded - ONEOFFICE_ACCESS_TOKEN: ✅ Có


In [4]:
# Kiểm tra cấu hình
print(f"GCP Project: {settings.gcp_project}")
print(f"GCP Region: {settings.gcp_region}")
print(f"Bronze Bucket: {settings.bronze_bucket}")
print(f"OneOffice Dataset: {settings.oneoffice_dataset}")
print(f"1Office Base URL: {settings.oneoffice_base_url}")

# Kiểm tra credentials
oneoffice_token_available = bool(settings.oneoffice_access_token)
print(f"\n1Office Access Token: {'✅ Có' if oneoffice_token_available else '⚠️ Không có'}")

if not oneoffice_token_available:
    print("\n⚠️ Chưa có 1Office Access Token!")
    print("   Có 2 cách để lấy:")
    print("   1. Set ONEOFFICE_ACCESS_TOKEN trong .env file")
    print("   2. Chạy cell bên dưới để lấy từ Secret Manager (cần GCP credentials)")


GCP Project: sync-nhanhvn-project
GCP Region: asia-southeast1
Bronze Bucket: sync-nhanhvn-project
OneOffice Dataset: oneoffice
1Office Base URL: https://minham.1office.vn/api

1Office Access Token: ✅ Có


## Khởi tạo Components

Khởi tạo extractor, loader và pipeline.


In [5]:
# Khởi tạo các components
# Lưu ý: Nếu đã lấy token từ Secret Manager ở cell trên, cần reload settings trước
from src.config import Settings
settings = Settings()  # Reload để lấy token mới nếu đã set trong env

extractor = OneOfficeExtractor()
loader = OneOfficeLoader()

# Khởi tạo pipeline
pipeline = OneOfficePipeline()

print("✅ Đã khởi tạo tất cả components thành công")


✅ Đã khởi tạo tất cả components thành công


## Chạy Pipeline

Chạy Daily Snapshot Pipeline để lấy dữ liệu và lưu snapshot cho ngày hôm nay.


In [6]:
# Chạy Daily Snapshot Pipeline (mặc định: hôm nay)
print("=" * 60)
print("Đang chạy OneOffice Daily Snapshot Pipeline")
print(f"Snapshot date: {date.today().isoformat()}")
print("=" * 60)

result = pipeline.run_daily_snapshot()

print("\n" + "=" * 60)
print("Kết quả Pipeline:")
print("=" * 60)
for key, value in result.items():
    print(f"{key}: {value}")
print("=" * 60)


2025-12-15 09:25:58,226 - src.features.one_office.pipeline - INFO - Starting Daily Snapshot Pipeline for 1Office...
2025-12-15 09:25:58,226 - src.features.one_office.pipeline - INFO - Starting Daily Snapshot Pipeline for 1Office...
2025-12-15 09:25:58,230 - src.features.one_office.components.extractor - INFO - Starting to fetch all personnel profiles from 1Office...
2025-12-15 09:25:58,230 - src.features.one_office.components.extractor - INFO - Starting to fetch all personnel profiles from 1Office...


Đang chạy OneOffice Daily Snapshot Pipeline
Snapshot date: 2025-12-15


2025-12-15 09:25:58,456 - src.features.one_office.components.extractor - INFO - Page 1: Fetched 100 items.
2025-12-15 09:25:58,456 - src.features.one_office.components.extractor - INFO - Page 1: Fetched 100 items.
2025-12-15 09:25:59,162 - src.features.one_office.components.extractor - INFO - Page 2: Fetched 100 items.
2025-12-15 09:25:59,162 - src.features.one_office.components.extractor - INFO - Page 2: Fetched 100 items.
2025-12-15 09:26:00,407 - src.features.one_office.components.extractor - INFO - Page 3: Fetched 100 items.
2025-12-15 09:26:00,407 - src.features.one_office.components.extractor - INFO - Page 3: Fetched 100 items.
2025-12-15 09:26:01,410 - src.features.one_office.components.extractor - INFO - Page 4: Fetched 100 items.
2025-12-15 09:26:01,410 - src.features.one_office.components.extractor - INFO - Page 4: Fetched 100 items.
2025-12-15 09:26:02,099 - src.features.one_office.components.extractor - INFO - Page 5: Fetched 100 items.
2025-12-15 09:26:02,099 - src.feature


Kết quả Pipeline:
status: success
extracted: 1062
loaded: 1062


## Test Từng Component Riêng Lẻ

Test từng component riêng lẻ để debug.


### Test Extractor

Test extractor để lấy dữ liệu từ 1Office API.


In [7]:
# # Test Extractor - Extract tất cả personnel
# print("Đang test Extractor - Fetch all personnel...")

# data = extractor.fetch_all_personnel()

# print(f"\nKết quả extract:")
# print(f"  - Total records: {len(data)}")
# if data:
#     print(f"  - Sample columns: {list(data[0].keys())[:10]}")
#     print(f"  - Sample record:")
#     import json
#     print(json.dumps(data[0], indent=2, ensure_ascii=False)[:500])


### Test Loader

Test loader để upload raw JSON lên GCS và load vào BigQuery.


In [8]:
# # Test Loader - Upload raw JSON và load vào BigQuery
# # Lưu ý: Cần có dữ liệu từ extractor trước
# print("Đang test Loader...")
# print("⚠️ Lưu ý: Loader sẽ tự động upload raw JSON lên GCS trước khi load vào BigQuery")

# # Extract data trước
# test_data = extractor.fetch_all_personnel()
# print(f"Extracted {len(test_data)} records")

# if test_data:
#     # Load vào BigQuery (tự động backup lên GCS)
#     count_loaded = loader.load_snapshots(test_data)
#     print(f"\nKết quả Loader:")
#     print(f"  - Records loaded: {count_loaded}")
#     print(f"  - BigQuery table: {loader.table_id}")
#     print(f"  - GCS backup: Đã tự động upload raw JSON")
# else:
#     print("⚠️ Không có dữ liệu để load. Chạy extractor trước.")
